# 목차
<hr/>

<ol>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">필요한 모듈 불러오기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 들어가며</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : 데이터의 형태</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 3</span> : 레이어는 어렵다?</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 4</span> : 딥러닝의 근본! Linear 레이어</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 정보를 집약시키자! Convolution 레이어</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 6</span> :  핵심만 추려서 더 넓게! Pooling 레이어</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 7</span> : 집약된 정보의 복원! Deconvolution 레이어</i></li>
</ol>

<div style = "height: 50px;"><div/>

# 필요한 모듈 불러오기

In [10]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 들어가며

## 학습목표
<hr/>

<ul>
	<li style = "font-size:15px; margin-bottom:10px;">레이어의 개념을 이해한다.</li>
	<li style = "font-size:15px; margin-bottom:10px;">딥러닝 모델 속 각 레이어(Linear, Convolution)의 동작 방식을 이해한다.</li>
	<li style = "font-size:15px; margin-bottom:10px;">데이터의 특성을 고려한 레이어를 설계하고, 이를 Tensorflow로 정의하는 법을 배운다.</li>
</ul>





딥러닝은 $y = Wx + b $에서 최적의 $W$ (Weight)와 b를 찾는 과정이다.<br/>
이를 딥러닝의 미싱 링크라 칭한다.

우리가 정의하는 수많은 신경망들은 각기 다른 형태의 Weight를 갖고 있고, 그마다 독특한 특성을 가지고 있습니다. 데이터에서 원하는 특징을 효과적으로 추출하기 위해선 올바른 Weight를 정의하는 과정이 중요합니다. 그리고 그 과정은 하나의 직관이 되어 신경망을 분석하거나 설계하는 데에 큰 도움이 될 것입니다.

우리는 데이터의 차원 변화를 좇으며 각기 다른 신경망들이 갖는 Weight의 특성을 살펴보고, 앞서 언급한 미싱 링크의 진정한 의미를 두 번에 걸쳐 알아볼 예정입니다. 이번 노드에서는 Linear 레이어와 Convolution 레이어를 집중적으로 공부하고, 다음번에는 Embedding 레이어와 Recurrent 레이어를 다루어 볼 것입니다. 이 과정을 통해 데이터의 차원 변화를 좇아 여러 신경망들이 갖는 Weight의 특성을 살펴보면서, 앞서 언급한 미싱 링크의 진정한 의미를 깨닫게 되기를 기대합니다.

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : 데이터의 형태

딥러닝을 이해하는 방법 중 가장 쉬운 방법은 데이터의 형태 변화를 좇는 것이다.<br/>
10개 단어의 문장을 `5개 단어로 요약`했다면 그것은 정보를 집약시킨 경우일 것이고,<br/> 
`20개 단어로 확장`했다면 그것은 정보를 더 세밀하게 표현한 경우이다.

<div style = "height: 50px;"><div/>

## 1. 매트릭스 예시
<hr/>

#### 기업 A에 지원한 지원자들의 인적 사항

<img src = "img/01.png">
<p style = "text-align:center; color:gray;">&lt;표 1&gt;</p>

<표 1> 의 데이터는 (3, 6) 의 매트릭스로 표현이 가능합니다. 만약 기업 B의 데이터가 추가된다면 어떨까요?

#### 기업 B의 데이터가 추가된 매트릭스

<img src = "img/02.png">
<p style = "text-align:center; color:gray;">&lt;표 2&gt;</p>

<표 2>의 데이터는 (2, 3, 6) 의 매트릭스로 표현이 가능합니다. 그럼 이미지 데이터는 어떨까요?

<div style = "height: 50px;"><div/>

## 2. 이미지 채널
<hr/>

이미지 데이터는 보통 `채널(Channel)` 이라는 것을 가집니다. 우리가 일상적으로 사용하는 것은 대부분 **RGB** 이미지인데요, 여기서 **RGB**는 **Red** 채널, **Green** 채널, 그리고 **Blue** 채널까지 `총 3개의 채널을 의미`합니다. 아래와 같은 분위기의 사진을 혹시 보신 적이 있나요?

<img src = "img/03.png" style = "width: 40%;">
<p style = "margin-bottom:50px; text-align:center; color:gray;">[Photo by Kyaw Tun on Unsplash]</p>

위 이미지는 **Red 채널만을 나타낸 이미지**이다.

<img src = "img/04.png" style = "width: 60%;">
<p style = "margin-bottom:50px; text-align:center; color:gray;">[Photo by Kyaw Tun on Unsplash]</p>

같은 맥락으로 Green 채널과 Blue 채널도 추출할 수 있고, 모든 채널을 한 데 겹치면 우리 눈에 익숙한 RGB 이미지가 된다.

<img src = "img/05.png" style = "width: 60%; margin-top:50px;">

RGB 이미지라고 가정을 한다면, 위 그림의 데이터는 `(3, 1920, 1080)` 또는 `(1920, 1080, 3)` 의 매트릭스로 표현이 가능합니다.<br/>
그리고 표현 방식에 따라 `Channel`, `Width`, `Height`의 이니셜로 **(C, W, H)**, **(W, H, C)**와 같이 표기합니다. 이 표기법을 알고 있다면, 어떤 딥러닝 모듈이 요구하는 데이터 형태를 알아보고 그에 맞게 데이터를 변형시킬 수 있습니다.

<div style = "height: 25px;"><div/>

<div style = "width: 80%; height:auto; padding:5%; border-style:solid; border-width: 2px; border-color:#71C5FF; border-radius:10px;">
<i style = "font-weight:bold;font-style: normal;">Q1. 해상도가 1280 x 1024(30fps)이고 러닝타임(T)이 90분인 흑백 영화 데이터는 어떻게 표현할 수 있을까요? 표기는 (T, C, H, W)를 따라주세요.</i><br/>
<p style = "margin-bottom:0px;">(162000, 1, 1024, 1280)<br/><br/>
Time: 90분 x 60초 x 30fps Channel: 흑백 영화는 단일 채널로 이루어진다. Height: 1024 Width: 1280</p>
</div>

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 3</span> : 레이어는 어렵다?

**레이어(layer)** 라는 단어는 제법 많은 곳에서 쓰이고 있습니다. 통신을 공부하셨다면 OSI 7 레이어를 들어보셨을 것이고, 포토샵에서도 레이어가 쓰이며, 심지어 레이어드 패션이 유행한 적도 있었죠. 약간씩 다른 의미로 사용되고 있는 용어지만, 우리가 이해해야 할 개념에 가장 가까운 정의는 다음과 같습니다.

> 하나의 물체가 여러 개의 논리적인 객체들로 구성되어 있는 경우, 이러한 각각의 객체를 하나의 레이어라 한다.

신경망이라는 물체를 구성하는 여러 개의 논리적인 레이어들을 이해하는 것은 곧 신경망 그 자체를 이해하는 것과 같죠. 서론의 Weight 얘기를 기억하나요? 신경망의 Weight라고 표현했지만, 정확히는 **레이어의 Weight**가 맞습니다. <br/><br/>
신경망은 레이어들의 각기 다른 Weight, 그것들이 유기적으로 연결되어 이뤄내는 하나의 결과물인 것이죠. 그것은 마치 인간의 두뇌와 닮아서 **인공 신경망**이라는 이름을 갖게 됩니다. 정말 신비하죠?

<img src = "img/06.png" style = "width:60%; margin-top:50px;">
<p style = "text-align:center; color:gray;">[▲ 대충 이런 모양새...? (Photo by Josh Riemer on Unsplash)]</p>

<div style = "height: 50px;"><div/>

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP41</span> : 딥러닝의 근본! Linear 레이어

[자세하고 수학적인 이해를 원하는 분을 위해 (자막 O)](https://youtu.be/kYB8IZa5AuE)

[쉽고 직관적인 이해를 원하는 분을 위해](https://youtu.be/vVvjYzFBUVk)

Linear 레이어는 선형 변환을 활용해 데이터를 특정 차원으로 변환하는 기능을 합니다. 100차원의 데이터를 300차원으로 변환한다면 데이터를 더 풍부하게 표현하는 효과가 있고, 반대로 10차원의 데이터로 변환한다면 데이터를 집약시키는 효과가 있습니다.

#### 예시

<img src = "img/07.png">

위 그림의 두 사각형은 모두 (x, y) 2차원의 점 4개로 표현 가능하므로, 각각 (4, 2) 행렬 형태의 데이터로 표현할 수 있습니다. 우리는 두 사각형을 각각 어떤 하나의 정수로 표현하고자 합니다. 실은 이 정수는 우리가 구분하고자 하는 사각형의 종류(class)를 의미합니다. 이를 위해, 데이터를 어떻게 집약시킬지 구상해 보겠습니다.

> <식1><br/>
> - 1단계: $(4, 2) ⋅ (2, 1) = (4, )$
> - 2단계: $(4, ) ⋅ (4, 1) = (1, )$

위 단계를 사용하면 각각의 사각형을, 정보가 집약된 하나의 정수로 표현할 수 있습니다. 2차원을 1차원으로 변환하는 데에 (2, 1) 행렬이 하나 선언되고, 4차원을 1차원으로 변환하는 데에 (4, 1) 행렬이 하나 선언됨에 유의합시다. <br/>
여기서 각각의 행렬들이 **Weight** 입니다. Linear 레이어는 `(입력의 차원, 출력의 차원)`에 해당하는 Weight를 가지는 특성을 가지고 있습니다.

<div style = "height: 25px;"><div/>

#### 코드로 표현하기

In [20]:
batch_size = 64

In [21]:
boxes = tf.zeros((batch_size, 4, 2))

Tensorflow는 Batch를 기반으로 동작하므로,<br/>사각형 2개 세트를 batch_size개만큼 만든 후 처리를 하게 됩니다.

<div style = "height: 25px;"><div/>

In [22]:
print('1단계 연산 준비 : ', boxes.shape)

1단계 연산 준비 :  (64, 4, 2)


In [23]:
first_linear = tf.keras.layers.Dense(units = 1, use_bias = False)


`units`은 `출력 차원 수`를 의미합니다.<br/>
Weight 행렬 속 실수를 인간의 뇌 속 하나의 뉴런 **유닛**취급을 하는 거죠!

<div style = "height: 25px;"><div/>

In [24]:
first_out = first_linear(boxes)
first_out.shape

TensorShape([64, 4, 1])

In [25]:
# 사이즈 변환
first_out = tf.squeeze(first_out, axis = -1)
first_out.shape

TensorShape([64, 4])

$(4, 1)을 (4,)$로 **변환**해줍니다. = 불필요한 차원 축소

<div style = "height: 25px;"><div/>

In [29]:
print("1단계 연산 결과:", first_out.shape)
print("\n1단계 Linear Layer의 Weight 형태:", first_linear.weights[0].shape)
print("\n2단계 연산 준비:", first_out.shape)

1단계 연산 결과: (64, 4)

1단계 Linear Layer의 Weight 형태: (2, 1)

2단계 연산 준비: (64, 4)


In [30]:
second_linear = tf.keras.layers.Dense(units = 1, use_bias = False)

In [32]:
second_out = second_linear(first_out)
second_out.shape

TensorShape([64, 1])

In [33]:
second_out = tf.squeeze(second_out, axis = -1)
second_out.shape

TensorShape([64])

In [35]:
print("2단계 연산 결과:", second_out.shape)
print("\n2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

2단계 연산 결과: (64,)

2단계 Linear Layer의 Weight 형태: (4, 1)


<div style = "height: 25px;"><div/>

<img src = "img/08.png">

하지만 순탄치 않습니다. 위 그림을 보니 두 사각형에 대해 1단계를 거치고 난 결과가 동일하군요. 이렇게 되면 <식 1>의 2단계 입력이 동일해지니 두 번째 (4, 1) Weight를 거치는 것이 의미가 없어집니다. <br/><br/>
여기서 모든 `Weight의 모든 요소`를 **Parameter** 라고 합니다.<br/><br/>
총 6개 (위 그림에서는 2개)의 Parameter로 이 문제를 해결하기엔 역부족이었던 것 같습니다.<br/>
첫 번째 접근은 데이터를 집약하는 데에만 집중했으니, 이번엔 데이터를 더 풍부하게 만들어 봅시다.

> <식2><br/>
> - 1단계: $(4, 2) ⋅ (2, 3) = (4, 3)$
> - 2단계: $(4, 3) ⋅ (3, 1) = (4, )$
> - 3단계: $(4, ) ⋅ (4, 1) = (1, )$ 

<img src = "img/09.png">

1단계의 결과로 각 사각형에 대해 독립적인 정보가 생겨나기 시작합니다. <br/>
예컨대 <식 2>는 첫 번째 접근에 비해 더 많은 사각형을 구분해낼 수 있을 것 같군요!



<div style = "width: 80%; height:auto; padding:5%; border-style:solid; border-width: 2px; border-color:#71C5FF; border-radius:10px;">
<i style = "font-weight:bold;font-style: normal;">Q2. &lt;식 2&gt;에서는 총 몇 개의 Parameter가 사용되었나요? (bias는 아직 배우지 않았기 때문에, 생각하지 않습니다!)</i><br/>
<p style = "margin-bottom:20px;">13개.<br/><br/>
1단계에서 2 x 3 = 6, 2단계에서 3 x 1 = 3, 3단계에서 4 x 1 = 4,<br/><br/>
총 6 + 3 + 4 = 13개의 Parameter가 사용되었다.</p>
</div>

이번에는 <식 2>의 과정을 Tensorflow 코드로 작성해 봅시다.
작성하며 데이터 차원의 변화와 각 Weight의 형태를 확인하고, `tf.keras.layers.Layer.count_params()` 함수를 인터넷에서 찾아본 후 활용하여 총 **parameter** 개수가 Q2의 정답과 일치하는지 체크해 보세요.

In [43]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2))

print("1단계 연산 준비:", boxes.shape)

########
# Step 1: (4,2)차원인 boxes를 (4,3)으로 확장시키는 Linear Layer를 활용하세요.
first_linear2 = tf.keras.layers.Dense(units = 3, use_bias = False)
first_out2 = first_linear2(boxes)


1단계 연산 준비: (64, 4, 2)


In [52]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2))

print("1단계 연산 준비:", boxes.shape)

########
# Step 1: (4,2)차원인 boxes를 (4,3)으로 확장시키는 Linear Layer를 활용하세요.

first_linear2 = tf.keras.layers.Dense(units = 3, use_bias = False) # units로 확장
first_out2 = first_linear2(boxes)

########

print("1단계 연산 결과:", first_out2.shape)
print("1단계 Linear Layer의 Weight 형태:", first_linear2.weights[0].shape)

print("\n2단계 연산 준비:", first_out2.shape)

# Dense = Linear
second_linear2 = tf.keras.layers.Dense(units=1, use_bias=False)
second_out2 = second_linear2(first_out2)
second_out2 = tf.squeeze(second_out2, axis=-1)

print("2단계 연산 결과:", second_out2.shape)
print("2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

print("\n3단계 연산 준비:", second_out2.shape)

########
# Step 2: 4차원인 second_out을 하나의 실수으로 집약시키는 Linear Layer를 활용하세요.

third_linear = tf.keras.layers.Dense(units = 1, use_bias = False) # units로 확장
third_out = third_linear(second_out2)
third_out = tf.squeeze(third_out, axis=-1)


########

print("3단계 연산 결과:", third_out.shape)
print("3단계 Linear Layer의 Weight 형태:", third_linear.weights[0].shape)

########
# Step 3: 모든 params를 더하여 total_parmams를 구하세요.

total_params = \
first_linear.count_params() + \
second_linear.count_params() + \
third_linear.count_params()

########

print("총 Parameters:", total_params)

1단계 연산 준비: (64, 4, 2)
1단계 연산 결과: (64, 4, 3)
1단계 Linear Layer의 Weight 형태: (2, 3)

2단계 연산 준비: (64, 4, 3)
2단계 연산 결과: (64, 4)
2단계 Linear Layer의 Weight 형태: (3, 1)

3단계 연산 준비: (64, 4)
3단계 연산 결과: (64,)
3단계 Linear Layer의 Weight 형태: (4, 1)
2
4
4
총 Parameters: 10


# <span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 들어가며

## 1. 조건
<hr/>

### 소제목

<div style = "height: 50px;"><div/>